# FormulaFS

Run a formula, handle side effects, output buffering, etc. Serve the output via FS interface.

In [1]:
from fs.base import FS
from fs.info import Info
from fs.enums import ResourceType
from fs.errors import DirectoryExpected
from io import BytesIO
from hashlib import sha256
import json
from DcelJSONHashEncoder import dumphash
from DcelJSONEncoder import dumpdcel

def sample_function(*args, **kwargs):
    print("FormulaFS sample function side-effect.")
    return "Sample function return value."

class FormulaFS(FS):
    """
    Serve the output of a command via FS API.
    
    Handle buffering.
    """
    def __init__(self, formula_dict):
        super().__init__()
        if(type(formula_dict) == dict):
            # set self._formula_dict before calling 
            # hash_of_formula_dict()
            self._formula_dict = formula_dict

            if 'fn' in formula_dict:
                self._formula_name = formula_dict['fn']
                self._formula_hash = self.hash_of_formula_dict()
            else:
                raise Exception("FormulaFS requires a function 'fn' in the initialization dict.")
            self._buffer = None
        else:
            raise TypeError('FormulaFS requires a dict to initialize.')
            
    def __repr__(self):
        return f"<FormulaFS({self._formula_dict})>"
    
    # ---- FS overloads ----
    
    def getinfo(self, path, namespaces=['basic']):
        nodetype = ResourceType.unknown
        return Info({
            'basic': {
                 'name': self._formula_name,
                 'is_dir': False
             },
        })
    
    def listdir(self, path):
        raise DirectoryExpected(path)
    
    def makedir(self, path, *args, **kwargs):
        raise ResourceInvalid(self._formula_name)
    
    def openbin(self, path, *args, **kwargs):
        if self.isdirty or (self._buffer == None):
            buf = self._formula_dict['fn'](*(self._formula_dict['args']))
            self.flushbuffer(buf)
        else:
            buf = self._buffer
        if not hasattr(buf, 'encode'):
            buf = dumpdcel(buf)
        return BytesIO(buf.encode())
    
    def remove(self, path, *args, **kwargs):
        raise ResourceInvalid(self._formula_name)
    
    def removedir(self, path, *args, **kwargs):
        raise ResourceInvalid(self._formula_name)
    
    def setinfo(self, path, *args, **kwargs):
        raise ResourceInvalid(self._formula_name)
    
    def hash(self, name=''):
        """Return hash of the dictionary that defines the formula."""
        if not hasattr(self, '_formula_hash'):
            _formula_hash = self.hash_of_formula_dict()
            self._formula_hash = _formula_hash
            return _formula_hash
        if self.isdirty:
            return self.hash_of_formula_dict()
        return self._formula_hash
    
    # ---- Iterator support ----
    
    def __iter__(self):
        if not hasattr(self._buffer, '__iter__'):
            return TypeError(f'Output from {self._formula_name} is not iterable.')
        # These are iterable.
        if type(self._buffer) == dict:
            # Allows conversion from dict to dict.
            return self._buffer.items().__iter__()
        return self._buffer.__iter__()
    
    # ---- String support ----
    
    def __str__(self):
        return self.readtext('')
    
    # ---- Introspection ----
    
    def __desc__(self):
        return self._formula_name
    
    # ---- Direct value access ----
    
    @property
    def value(self):
        if self._buffer == None:
            buf = self._formula_dict['fn'](*(self._formula_dict['args']))
            self.flushbuffer(buf)
        else:
            buf = self._buffer
        return buf
            
    # ---- Checksum support ----
    def hash_of_formula_dict(self):
        return dumphash(self._formula_dict)
    
    @property
    def isdirty(self):
        return self._formula_hash != self.hash_of_formula_dict()
    
    # ---- Buffer ----
    def flushbuffer(self, buf):
        self._buffer = buf
        self._formula_hash = self.hash_of_formula_dict()
    

In [2]:
# Setup for testing the fstab parser.
from HienaMP import hiena_mp
from fstab_hg import fstab_hg
from cskvp_hg import cskvp_hg
from fs import open_fs
from Dcel import Dcel
from DictFS import DictFS

In [3]:
# Setup the fstab parser data.
fs_dcel = Dcel('demo-files/fs', service_class=open_fs)
file_dcel = fs_dcel['@']['etc']['fstab']
formula_dcel = Dcel({'fn':hiena_mp, 'args':[fstab_hg, file_dcel]},
                    service_class=FormulaFS)
walker_dcel = Dcel(formula_dcel.value, service_class=DictFS)

In [4]:
print(walker_dcel.inspect())

address: <class 'str'>:/
        abspath: <class 'str'>:/
        service: <class 'DictFS.DictFS'>:<DictFS.DictFS object at 0x7f8e506cfdc0>
        value: <class 'dict'>:{'1': {'spec': <Dcel.Dcel object at 0x7f8e5061cdf0>, 'file': <Dcel.Dcel object at 0x7f8e5061cdc0>, 'vfstype': <Dcel.Dcel object at 0x7f8e5061f1c0>, 'mntopts': <Dcel.Dcel object at 0x7f8e5061c160>, 'freq': <Dcel.Dcel object at 0x7f8e5061cf10>, 'passno': <Dcel.Dcel object at 0x7f8e5061c220>}, '2': {'spec': <Dcel.Dcel object at 0x7f8e5061f0d0>, 'file': <Dcel.Dcel object at 0x7f8e5061f040>, 'vfstype': <Dcel.Dcel object at 0x7f8e5061f100>, 'mntopts': <Dcel.Dcel object at 0x7f8e5061f130>, 'freq': <Dcel.Dcel object at 0x7f8e5061f070>, 'passno': <Dcel.Dcel object at 0x7f8e5061f0a0>}, '3': {'spec': <Dcel.Dcel object at 0x7f8e5061ef80>, 'file': <Dcel.Dcel object at 0x7f8e5061eef0>, 'vfstype': <Dcel.Dcel object at 0x7f8e5061efb0>, 'mntopts': <Dcel.Dcel object at 0x7f8e5061efe0>, 'freq': <Dcel.Dcel object at 0x7f8e5061ef20>, 'pass

In [5]:
subfile_dcel = walker_dcel['1']['mntopts']
subformula_dcel = Dcel({'fn':hiena_mp, 'args':[cskvp_hg, subfile_dcel]},
                      service_class=FormulaFS)
subwalker_dcel = Dcel(subformula_dcel.value, service_class=DictFS)

In [6]:
# Pre-flight Check
print(subwalker_dcel.inspect())

address: <class 'str'>:/
        abspath: <class 'str'>:/
        service: <class 'DictFS.DictFS'>:<DictFS.DictFS object at 0x7f8e506ce6e0>
        value: <class 'dict'>:{'user': <Dcel.Dcel object at 0x7f8e5061e920>, 'shortid': <Dcel.Dcel object at 0x7f8e5061e6b0>, 'idcard': <Dcel.Dcel object at 0x7f8e5061e9b0>}
        _map: <class 'NoneType'>:None
        _dir: <class 'NoneType'>:None
        


In [15]:
print(subwalker_dcel.getinfo('shortid'))
print(subwalker_dcel['shortid'])

<dir 'shortid'>
root


In [8]:
shortid_dcel = subwalker_dcel['shortid']

In [9]:
print(shortid_dcel.inspect())

address: <class 'str'>:/shortid
        abspath: <class 'str'>:/shortid
        service: <class 'DictFS.DictFS'>:<DictFS.DictFS object at 0x7f8e506ce6e0>
        value: <class 'Dcel.Dcel'>:root
        _map: <class 'NoneType'>:None
        _dir: <class 'NoneType'>:None
        


In [19]:
# Test: __init__()
my_formula = FormulaFS({'fn':sample_function, 'args':['Arg hello.']})

In [24]:
# Test: gettext or readtext
# The first run of gettext() should execute the function,
# any side effects will happen here.
print(my_formula.gettext(''))

FormulaFS sample function side-effect.
Sample function return value.


In [20]:
# Test: hash value
print(my_formula.hash('.'))

3cce2092a5604cb797cdd3fd2a316cbdc158f06b4926b75a3c59848f7f418dfe


In [19]:
# Test: Modify formula dict
my_formula._formula_dict['args'].append(['Merry Christmas.'])

In [21]:
# Test: isdirty
my_formula.isdirty

True

In [22]:
# Test isdir()
my_formula.isdir('')

False

In [23]:
# Test getinfo()
info = my_formula.getinfo('.')
print(info.raw)
print(info.name)

{'basic': {'name': <function sample_function at 0x7f0cf572aa70>, 'is_dir': False}}
<function sample_function at 0x7f0cf572aa70>


In [25]:
# The second run of gettext should use the buffered output.
print(my_formula.gettext(''))

Sample function return value.


In [26]:
# Subsequent runs also use the buffer.
print(my_formula.gettext(''))

Sample function return value.


In [27]:
# When the formula dictionary is updated, `.isdirty` will be True.
print(my_formula.isdirty)
my_formula._formula_dict['args'].append(['Feliz Neuvos Anos.'])
print(my_formula.isdirty)

False
True


In [12]:
# If `isdirty` is True, the next invocation of `gettext()` will execute the function.
# This will, again, create side effects.
print(my_formula.gettext(''))

FormulaFS sample function side-effect.
Sample function return value.


In [29]:
# And once more, subsequent `gettext()` calls will use the buffer
# until `isdirty` becomes True again.
print(my_formula.gettext(''))

Sample function return value.
